This is a simple calculator that you can use to estimate your cumulative covid19 mortality risk over multiple infections spanning several years, based on current CDC mortality estimates.

Enter your current ```age```, the number of ```years``` and the number of infections per year (```infections_per_year```) to model.  For example, to model the mortality risk for a 35 year old that expects to be reinfected an average of twice per year over the next ten years, in the cell labeled ```2 - PARAMETERS``` you'd enter  
```python
    age = 35
    years = 10
    infections_per_year = 2
```
Using the CDC's estimates of Infection Fatality Rate, this function will calculate your updated mortality risk per infection and return an estimate of your overall mortality risk for the given time period.  Since this calculator is based on CDC estimates of IFR, which is based on historical data, it doesn't take into account variations in mortality per infection by new variants, nor does it take into account as-yet undiscovered pharmaceutical interventions.  

Moreover, since infection sometimes results in chronic conditions specifically associated with worse outcomes and often results in some degree of immune "aging", neither of which are modelled here, this app is, barring new pharmaceutical interventions, likely to underestimate risk to some degree, and the results calculated are probably best thought of as a "best case scenario" for the parameters modelled.

This calculator does, however, take into account the protective effect of previous infection - meaning it assumes that after you recover from your first infection, your risk of dying from subsequent infections is reduced by 39%.  This is based on the protective effect of infection observed thus far in the pandemic, and may once again lead to an underestimate of risk (if, for example, this effect doesn't endure, if new variants completely evade immunity, or if Antibody Dependent Enhancement becomes an issue).

Scroll down if you'd like to read about how these estimates are calculated. Otherwise, follow the instructions in cells ```1```-```3``` below.

**BUT FIRST** a note on infections_per_year. Prior to omicron, it had been estimated that recovered individuals could, on average, be reinfected approximately every 18 months. Since omicron, however, case reports of reinfections just months, and sometimes weeks, after recovery have become common.  If you're looking to model a variety of plausible scenarios for a future without NPIs, you might want to look at mortality estimates over a given time frame with ```infections_per_year``` set to values between 2/3 and 4.

### 1 - CODE
Highlight the cell below and type shift + enter

In [13]:
def calc_mortality(age, years, infections_per_year, init_ifr=0, adjust_ifr_for_age=1.11, pct_protect=.61):
    if init_ifr == 0:
        ifr_dict = {20+i: .0005*(1.11**i) for i in range(1, 46)}
        # ifr ranges from 0.0005 for a 20 year old to 0.055 for a 65 year old
    l = [ifr_dict[age], ifr_dict[age]*pct_protect]
    for i in range(1, years+1):
        l.append(1-(1-(ifr_dict[age+i]*pct_protect))**infections_per_year)
    return sum(l)

### 2 - PARAMETERS 
Enter your ```age``` and a value for ```years``` and ```infections_per_year``` in the cell below, then 
type shift + enter

In [15]:
age = 35
years = 10
infections_per_year = 2

### 3 - CALL
Highlight the cell below and type shift + enter

In [12]:
calc_mortality(age, years, infections_per_year)

0.05794513887818058

### EXPLANATION

This calculator works like this:  if you are relatively young and healthy, the likelihood that you'll end up dying from a covid infection is relatively small (on the order of 0.0005), and the mortality risk after $x$ number of infections is   

$0.0005 + (1-((1-(0.0005*0.61))^{x-1}))$   

where 0.61 is the factor that represents the 39% reduction in mortality risk conferred from a previous infection.  So, for a relatively young, healthy person, the cumulative mortality risk of, for example, 6 infections, is   

$0.0005 + (1-(1-(0.0005)*.61)^5) = 0.002$

Now, since age is likely the most significant factor in determining the mortality risk for most people, and since the risk of mortality increases considerably as we age (from about 0.0005 at age 18 to about 0.09 for the 65+ age group), in order to estimate your risk of mortality over $x$ number of years, we have to account for the fact that your mortality risk will gradually increase over those $x$ years.  So, for the example above, if those six infections happened over a three year period, the calculation would look like this:  

First year mortality risk: $0.0005 + (0.0005*0.61) = 0.0008$    
Second year mortality risk: $1-(1-(0.00055)*.61)^2 = 0.00067$  
Third year mortality risk: $1-(1-(0.00062)*.61)^2 = 0.00076$  

Notice that the IFR, which was 0.0005 during the first year, gradually increased to 0.00062 over the next two years. Also, notice that the first year was the riskiest thus far. This reflects the fact that, historically, recovery from an initial infection has imparted some protection against severe outcomes from subsequent infections.  Finally, notice how the risk begins creeping back up in year three as a result of the gradually increasing IFR.

To explain how to calculate the overall three-year mortality risk for this example, it might help to use a more familiar example.  If you roll 3 dice, what are the chances that none of the three dice is a 6?  We know that the odds of rolling a six are 1/6 for each die rolled, but what about the odds of getting at least one 6 when you roll three dice? It turns out that the odds of getting at least one 6 when you roll three dice is equal to the sum of the following probabilities:   
  
One six: $\binom{3}{1} \frac{1}{6}*\frac{5}{6}^2$  
Two sixes: $\binom{3}{2} \frac{1}{6}^2*\frac{5}{6}$  
Three sixes: $\binom{3}{3} \frac{1}{6}^3$  
  
Which is equal to $\sum_{k=1}^3 \binom{3}{k} \frac{1}{6}^{k}*\frac{5}{6}^{3-k}$ or one minus the probability of rolling zero sixes: $1-(\frac{5}{6}*\frac{5}{6}*\frac{5}{6})$. So, to calculate the cumulative mortality rate over three years, we multiply (1-0.0008), (1-0.00067) and (1-0.00076), and subtract the total from one      

$1-((1-0.0008)*(1-0.00067)*(1-0.00076)) = 0.002228$
  
and get a cumulative, IFR-adjusted mortality risk for 6 infections over three years of 0.00223, or about 0.223%.